In [1]:
#| default_exp core

# core

> Tiny helper to authenticate Claudette to Vertex AI

In [1]:
#| export
import json, os
from pathlib import Path
from fastcore.utils import *

from anthropic import AnthropicVertex, AsyncAnthropicVertex
from claudette import Client, AsyncClient


## Optional: Getting a GC Service Account Key File (SAKF)

If you do not already have a Google CLoud Service Account Key File (SAKF), but you are already authenticated via Google cloud, these are the steps for getting a SAKF by using the Google Cloud API. 

### Creating a GC service account

In [30]:
from IPython.display import Markdown
from playwrightnb import url2md,read_page
from ContextKit import read_gist

In [3]:
sak_docs = read_gist('https://gist.github.com/jph00/3d12ac8ac76c7e7b3ef62dc889284838')

In [4]:
# !pip install -q google-cloud-service-usage

In [5]:
%%ai 0 -c
I want to create a service account key file using the Python API documented in $`sak_docs`. Can you read the docs and confirm that all the needed info is there, before we begin?

1. We'll need to use the `google.cloud.iam_admin_v1` client library
2. The documentation notes that we'll need:
   - project_id: The Google Cloud project ID
   - account: The service account ID or email

In [6]:
#| export
import google.auth
from google.cloud import iam_admin_v1
from google.cloud.iam_admin_v1 import types
from google.cloud import resourcemanager_v3
from google.iam.v1 import policy_pb2

We can work with Google's IAM API by using a client, assuming you have first run `gcloud auth application-default login`.

In [8]:
cli = iam_admin_v1.IAMClient()
credentials, project_id = google.auth.default()
project = f"projects/{project_id}"
project

'projects/jph001'

We will need a "service account" with the appropriate permissions. You can check your account list like so:

In [9]:
accounts = cli.list_service_accounts(name = project)
# accounts

...and here is how to create an account:

In [10]:
account_id="aiservice"
display_name="Vertex AI Service Account"
description="Access Vertex AI"

In [11]:
svc = dict(display_name=display_name, description=description)
account = cli.create_service_account(name=project, account_id=account_id, service_account=svc)
# account

In [13]:
polcli = resourcemanager_v3.ProjectsClient()
policy = polcli.get_iam_policy(resource=project)

In [15]:
member = f"serviceAccount:{account.email}"
roles = [ "roles/aiplatform.user", "roles/servicemanagement.quotaViewer", "roles/servicemanagement.quotaAdmin" ]

In [19]:
for role in roles:
    binding = policy_pb2.Binding()
    binding.role = role
    binding.members.append(member)
    policy.bindings.append(binding)
    
polres = polcli.set_iam_policy(request={"resource": project, "policy": policy})

### Creating a GC Service Account Key File from service account

In [21]:
key = cli.create_service_account_key(name = f"projects/{project_id}/serviceAccounts/{account.email}")

In [34]:
keyd = json.loads(key.private_key_data.decode())
keyd['region'] = 'us-east5'
keyb = json.dumps(keyd).encode()

In [36]:
path = Path('service-account-key.json')
path.write_bytes(keyb)

2331

In [52]:
nk = json.loads(path.read_text())
# nk

## Creating a vertexauth "superkey" to use for authentication

A vertexauth "superkey" is just SAKF file with key/value pair added for the region, so that this file can be the ONLY resource a needs to install to use this library.

If someone has given you a superkey, save it in `~/.config/vertexauth/default/superkey.json`.

If you only have a SAKF file, save a superkey by calling the following function

In [6]:
#| default_exp core

SUPERKEY_DEFAULT_PATH = Path.home() / ".config" / "vertexauth" / "default" / "superkey.json"

def save_superkey_file(SAKF_path, region) -> Path:
    d = json.loads(Path(SAKF_path)).read_text()
    d["region"] = region
    SUPERKEY_DEFAULT_PATH.parent.mkdir(parents=True,exists_ok=True)
    SUPERKEY_DEFAULT_PATH.write_text(json.dumps(d))
    path.chmod(0o600)
    return path

## Authenticating to Claudette

In [7]:
#| default_exp core

def get_anthropic_client(asink=False,anthropic_kwargs=None):
    d = json.loads(SUPERKEY_DEFAULT_PATH.read_text())
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(SUPERKEY_DEFAULT_PATH)
    os.environ["GOOGLE_CLOUD_PROJECT"]           = d['project_id']
    anthropic_kwargs = anthropic_kwargs or {}
    AV = AnthropicVertex if not asink else AsyncAnthropicVertex
    return AV(region=d['region'],project_id=d['project_id'], **anthropic_kwargs)    

def get_claudette_client(vertex_model='claude-3-5-sonnet-v2@20241022', 
                         asink=False, anthropic_kwargs=None):
    vertex_cli = get_anthropic_client(asink, anthropic_kwargs)
    if asink: return AsyncClient(vertex_model, vertex_cli)
    else: return Client(vertex_model, vertex_cli)

## Using claudette

In [8]:
cl = get_claudette_client()

In [9]:
cl('hi')

Hello! How can I help you today?

<details>

- id: `msg_vrtx_01VR4dF59fxMfGgxtMoXQ6jj`
- content: `[{'text': 'Hello! How can I help you today?', 'type': 'text'}]`
- model: `claude-3-5-sonnet-v2-20241022`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'input_tokens': 8, 'output_tokens': 12}`

</details>

In [84]:
# cli.delete_service_account(name=f"projects/{project_id}/serviceAccounts/aiservice@jph001.iam.gserviceaccount.com")

## Todo: quota management

In [29]:
quota_docs = read_gist('https://gist.github.com/jph00/943c51623abfe0deae65cfad2d821169')
svcuse_docs = read_gist('https://gist.github.com/jph00/042580724e98ae0cce2db50de92abd1b')

In [41]:
from google.cloud import service_usage_v1

In [64]:
%%ai 0 -c
I want to make sure we have all the quotas we need. See the docs in $`quota_docs` for examples of the quota API, and $`svcuse_docs` for service usage API docs. Do you see anything in the docs which might help here?

Yes I see some options we could use -- do you want me to outline them now?

In [75]:
scli = service_usage_v1.ServiceUsageClient()
services = scli.list_services(request={"parent": project, "filter":"state:ENABLED"})

## export -

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()